<a href="https://colab.research.google.com/github/Requenamar3/Data-Mining/blob/main/DM_Day1_DB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [ ]:
# File location and type
file_location = "/FileStore/tables/OnlineRetail.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df)

NameError: name 'spark' is not defined

In [ ]:
display(df.limit(5))

In [ ]:
df.printSchema()

In [ ]:
df.describe().show()

In [ ]:
# This line of code is a "trick"... it will convert your Spark DF - into a Pandas DF so normal "Python" rules apply
df = df.toPandas()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
# Drop Null values
df.dropna(subset=['CustomerID'], inplace=True)

# Keep positive quantity values
df = df.loc[df['Quantity'] > 0]

# Remove rows with negative price
df = df.loc[df['UnitPrice'] > 0]

# Ignore the records for the incomplete month
df = df.loc[df['InvoiceDate'] < '2011-12-01']

# Calculate total sales
df['Sales'] = df['Quantity'] * df['UnitPrice']

# Create a dataframe with orders
df_orders = df.groupby(['CustomerID',
                        'InvoiceNo']).agg({'Sales': sum,
                                           'InvoiceDate': max})

In [ ]:
df_orders.head()

In [ ]:
# Create a view or table

temp_table_name = "OnlineRetail_csv"

df.createOrReplaceTempView(temp_table_name)

In [ ]:
%sql

/* Query the created temp table in a SQL cell */

select * from `OnlineRetail_csv`

In [ ]:
# With this registered as a temp view, it will only be available to this particular notebook. If you'd like other users to be able to query this table, you can also create a table from the DataFrame.
# Once saved, this table will persist across cluster restarts as well as allow various users across different notebooks to query this data.
# To do so, choose your table name and uncomment the bottom line.

permanent_table_name = "OnlineRetail_csv"

# df.write.format("parquet").saveAsTable(permanent_table_name)